# ЗАДАНИЕ 4. Анализ поездок и Spark GraphFrames

In [41]:
import pyspark
from pyspark.sql import SparkSession, Row

In [42]:
packages = "graphframes:graphframes:0.6.0-spark2.3-s_2.11"

In [43]:
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from geopandas.tools import sjoin

In [44]:
import folium
from folium.plugins import HeatMap, HeatMapWithTime

In [45]:
conf = pyspark.SparkConf() \
        .set("spark.executor.memory", "1g") \
        .set("spark.executor.core", "2") \
        .set("spark.jars.packages", "graphframes:graphframes:0.6.0-spark2.3-s_2.11")\
        .setAppName("moviewRecomApp")

In [46]:
spark = SparkSession \
    .builder \
    .appName("tripApp") \
    .config(conf=conf) \
    .getOrCreate()

In [47]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType, TimestampType

In [48]:
import graphframes as gf

In [60]:
trip_schema = StructType([
    StructField(name="tripduration", dataType=IntegerType(), nullable=True),
    StructField("starttime", TimestampType(), True),
    StructField("stoptime", TimestampType(), True),
    StructField("start_station_id", IntegerType(), True),
    StructField("start_station_name", StringType(), True),
    StructField("start_station_latitude", DoubleType(), True),
    StructField("start_station_longitude", DoubleType(), True),
    StructField("end_station_id", IntegerType(), True),
    StructField("end_station_name", StringType(), True),
    StructField("end_station_latitude", DoubleType(), True),
    StructField("end_station_longitude", DoubleType(), True),
    StructField("bike_id", IntegerType(), True),
    StructField("usertype", StringType(), True),
    StructField("birth_year", IntegerType(), True),
    StructField("gender", IntegerType(), True)])

In [77]:
trips_data_path = "data/201902-citibike-tripdata.csv"
trips = spark.read.load(trips_data_path, format="csv", header="true", schema=trip_schema, inferSchema="false", sep=",")

print("Количество маршрутов:", trips.count())
trips.show(5)

Количество маршрутов: 943744
+------------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+----------+------+
|tripduration|           starttime|            stoptime|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id|  usertype|birth_year|gender|
+------------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+----------+------+
|         219|2019-02-01 00:00:...|2019-02-01 00:03:...|            3494|E 115 St & Lexing...|             40.797911|               -73.9423|          3501|E 118 St & Madiso...|          40.8014866

In [78]:
trips.printSchema()

root
 |-- tripduration: integer (nullable = true)
 |-- starttime: timestamp (nullable = true)
 |-- stoptime: timestamp (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: double (nullable = true)
 |-- start_station_longitude: double (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: double (nullable = true)
 |-- end_station_longitude: double (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- usertype: string (nullable = true)
 |-- birth_year: integer (nullable = true)
 |-- gender: integer (nullable = true)



## Задача 1

**1. определите для каждой станции количество начала поездок и количество завершения поездок**

In [84]:
from pyspark.sql.functions import desc,split,explode,count,avg,coalesce

stations = trips\
    .select(F.col('start_station_id').alias('id'), 
            F.col('start_station_name').alias('name'),
            F.col("start_station_latitude").alias('lat'), 
            F.col("start_station_longitude").alias('lng'))\
    .filter('lat is not null and lng is not null')\
    .distinct()
print('Количество станций: {}'.format(stations.count()))
print('Станции:')
stations.show()

print('Подсчет коиличеств поездок...')
stations = stations\
    .join(trips, stations.id == trips.start_station_id, how='left')\
    .groupBy('id', 'name', 'lat', 'lng') \
    .agg({'*': 'count'})\
    .withColumnRenamed('count(1)', 'start_cnt')\
    .join(trips, stations.id == trips.end_station_id, how='left')\
    .groupBy('id', 'name', 'lat', 'lng', 'start_cnt') \
    .agg({'*': 'count'})\
    .withColumnRenamed('count(1)', 'end_cnt')
stations.show()

Количество станций: 767
Станции:
+----+--------------------+-----------------+------------------+
|  id|                name|              lat|               lng|
+----+--------------------+-----------------+------------------+
| 312|Allen St & Stanto...|        40.722055|        -73.989111|
|3509|Lenox Ave & W 115 St|       40.8011939|       -73.9500739|
| 351|Front St & Maiden Ln|      40.70530954|      -74.00612572|
|3419| Douglass St & 4 Ave|       40.6792788|      -73.98154004|
| 259|South St & Whiteh...|      40.70122128|      -74.01234218|
|2017|     E 43 St & 2 Ave|      40.75022392|      -73.97121414|
| 504|     1 Ave & E 16 St|      40.73221853|      -73.98165557|
|3242|Schermerhorn St &...|40.69102925677968|-73.99183362722397|
|3491|    E 118 St & 1 Ave|         40.79747|         -73.93504|
|3664|North Moore St & ...|40.72019521437465| -74.0103006362915|
|3260|Mercer St & Bleec...|40.72706363348306|-73.99662137031554|
|3059|Pulaski St & Marc...|       40.6933982|        -73.

2. сопоставьте станции с кварталами города (zones) и определите суммы количества начала и завершения для кажого квартала выведите по убыванию и отобразите в виде картограмм (Choropleth)

In [95]:
from IPython.display import IFrame

def embed_map(m):
    m.save('index.html')
    return IFrame('index.html', width='100%', height='750px')

# выводим станции на карту
m = folium.Map()
for index, row in stations.toPandas().iterrows():
    folium.CircleMarker(location=(row["lat"], row["lng"]),
                        weight=1,
                        radius= 4,
                        color="#E3170A",
                        fill_color="#E3170A",
                        fill_opacity=0.3,
                        fill=True).add_to(m)
m.fit_bounds(m.get_bounds())
embed_map(m)

In [96]:
borough_data_path = "data/NYC Taxi Zones.geojson"

style_function = lambda x: {
    "color" : "#7EBDC3",
    "weight": 1
}

# выводим зоны на карту
folium.GeoJson(borough_data_path, name="geojson", style_function=style_function).add_to(m)
m.fit_bounds(m.get_bounds())
embed_map(m)

`Convert the zones GeoJson to GeoDataFrame:`

In [97]:
with open(borough_data_path) as f:
    zones_geojson = json.load(f)

column_name_list = [key for key, value in zones_geojson["features"][0]["properties"].items()]
column_name_list += ["geometry"]
column_name_list

['shape_area',
 'objectid',
 'shape_leng',
 'location_id',
 'zone',
 'borough',
 'geometry']

In [98]:
def get_pandas_rows(features):
    for item in features:
        row = list()
        for key, value in item["properties"].items():
            row.append(value)        
        polygons = list()
        for polygon in item["geometry"]["coordinates"]:
            polygons.append(Polygon(polygon[0]))
        row.append(MultiPolygon(polygons=polygons))
        yield row

In [99]:
zones = pd.DataFrame(get_pandas_rows(zones_geojson["features"]), columns=column_name_list)
zones.head(5)

,shape_area,objectid,shape_leng,location_id,zone,borough,geometry
0,0.0007823067885,1,0.116357453189,1,Newark Airport,EWR,(POLYGON ((-74.18445299999996 40.6949959999999...
1,0.00486634037837,2,0.43346966679,2,Jamaica Bay,Queens,(POLYGON ((-73.82337597260663 40.6389870471767...
2,0.000314414156821,3,0.0843411059012,3,Allerton/Pelham Gardens,Bronx,(POLYGON ((-73.84792614099985 40.8713422339999...
3,0.000111871946192,4,0.0435665270921,4,Alphabet City,Manhattan,(POLYGON ((-73.97177410965318 40.7258212813370...
4,0.000497957489363,5,0.0921464898574,5,Arden Heights,Staten Island,(POLYGON ((-74.17421738099989 40.5625680859999...


In [100]:
gdf_zones = gpd.GeoDataFrame(zones, geometry=zones["geometry"])
gdf_zones.head(5)

,shape_area,objectid,shape_leng,location_id,zone,borough,geometry
0,0.0007823067885,1,0.116357453189,1,Newark Airport,EWR,"MULTIPOLYGON (((-74.18445 40.69500, -74.18449 ..."
1,0.00486634037837,2,0.43346966679,2,Jamaica Bay,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."
2,0.000314414156821,3,0.0843411059012,3,Allerton/Pelham Gardens,Bronx,"MULTIPOLYGON (((-73.84793 40.87134, -73.84725 ..."
3,0.000111871946192,4,0.0435665270921,4,Alphabet City,Manhattan,"MULTIPOLYGON (((-73.97177 40.72582, -73.97179 ..."
4,0.000497957489363,5,0.0921464898574,5,Arden Heights,Staten Island,"MULTIPOLYGON (((-74.17422 40.56257, -74.17349 ..."


In [108]:
# объединяем с таблицей зон
points = gpd.GeoDataFrame(stations, geometry=gpd.points_from_xy(stations.lng, stations.lat))
stations_joined = sjoin(points, gdf_zones, how="left")
stations_joined

,id,name,lat,lng,start_cnt,end_cnt,geometry,index_right,shape_area,objectid,shape_leng,location_id,zone,borough
0,471,Grand St & Havemeyer St,40.712868,-73.956981,870,873,POINT (-73.95698 40.71287),257,0.000172309184842,255,0.0623841997664,255,Williamsburg (North Side),Brooklyn
1,496,E 16 St & 5 Ave,40.737262,-73.992390,3275,3301,POINT (-73.99239 40.73726),234,0.0000731054382894,234,0.0360721994984,234,Union Sq,Manhattan
2,3175,W 70 St & Amsterdam Ave,40.777480,-73.982886,1786,1761,POINT (-73.98289 40.77748),146,0.00015109426901,143,0.0541798538849,143,Lincoln Square West,Manhattan
3,243,Fulton St & Rockwell Pl,40.688226,-73.979382,588,824,POINT (-73.97938 40.68823),97,0.000163303970435,97,0.0624760147423,97,Fort Greene,Brooklyn
4,392,Jay St & Tech Pl,40.695065,-73.987167,1034,1101,POINT (-73.98717 40.69506),64,0.000081803882541,65,0.0446070683658,65,Downtown Brooklyn/MetroTech,Brooklyn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762,3053,Marcy Ave & Lafayette Ave,40.690081,-73.947915,180,158,POINT (-73.94791 40.69008),21,0.000322957654799,17,0.093522632948,17,Bedford,Brooklyn
763,3245,NYCBS DEPOT - DELANCEY,40.716444,-73.982331,2,20,POINT (-73.98233 40.71644),232,0.000216049973456,232,0.0614709085331,232,Two Bridges/Seward Park,Manhattan
764,3376,E 65 St & 2 Ave,40.764719,-73.962221,818,831,POINT (-73.96222 40.76472),142,0.0000766545579019,141,0.0415144638712,141,Lenox Hill West,Manhattan
765,3523,24 Ave & 29 St,40.772900,-73.916142,129,133,POINT (-73.91614 40.77290),225,0.00076436070058,223,0.166021925275,223,Steinway,Queens


In [111]:
# выводим суммы
count_for_zone = stations_joined\
    .groupby(["location_id", "zone"])\
    ['start_cnt', 'end_cnt'].sum()\
    .reset_index()
count_for_zone.sort_values(['start_cnt', 'end_cnt', 'location_id', 'zone'], ascending=False)

,location_id,zone,start_cnt,end_cnt
85,79,East Village,43333,43170
81,68,East Chelsea,38967,39395
27,170,Murray Hill,32572,32578
49,234,Union Sq,30527,31464
4,113,Greenwich Village North,26485,27054
...,...,...,...,...
78,62,Crown Heights South,676,712
42,226,Sunnyside,607,621
86,8,Astoria Park,400,398
43,228,Sunset Park West,397,484


Отображаем на карте сначала по количеству начала поездок, затем по количеству окончания поездок

In [112]:
m = folium.Map()

folium.Choropleth(
    geo_data=zones_geojson,
    data=df_count_for_zone,
    columns=["location_id", "begin count"],
    name="Number of started trips",
    legend_name="Number of started trips",
    key_on="feature.properties.location_id",
    highlight=True,
    nan_fill_color="grey",
    nan_fill_opacity=0.1,
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
).add_to(m)
m.fit_bounds(m.get_bounds())
embed_map(m)

In [116]:
m = folium.Map()

folium.Choropleth(
    geo_data=zones_geojson,
    data=df_count_for_zone,
    columns=["location_id", "end count"],
    name="Количества высадок",
    legend_name="Количества высадок",
    key_on="feature.properties.location_id",
    highlight=True,
    nan_fill_color="grey",
    nan_fill_opacity=0.1,
    fill_color="YlOrRd",
    fill_opacity=0.5,
    line_opacity=0.3,
).add_to(m)
m.fit_bounds(m.get_bounds())
embed_map(m)

## Задача 2
1. определите "важность" вершин графа поездок с использование PageRank

In [121]:
stations = trips\
    .select(F.col('start_station_id').alias('id'), 
            F.col('start_station_name').alias('name'),
            F.col("start_station_latitude").alias('lat'), 
            F.col("start_station_longitude").alias('lng'))\
    .filter('lat is not null and lng is not null')\
    .distinct()
stations.show()

edges = trips\
    .select(F.col("start_station_id").alias("src"), 
            F.col("end_station_id").alias("dst"), 
            "tripduration")\
    .dropna(subset=['src'])\
    .dropna(subset=['dst'])

print("Total number of trips:", df_edges.count())
df_edges.show()

+----+--------------------+-----------------+------------------+
|  id|                name|              lat|               lng|
+----+--------------------+-----------------+------------------+
| 312|Allen St & Stanto...|        40.722055|        -73.989111|
|3509|Lenox Ave & W 115 St|       40.8011939|       -73.9500739|
| 351|Front St & Maiden Ln|      40.70530954|      -74.00612572|
|3419| Douglass St & 4 Ave|       40.6792788|      -73.98154004|
| 259|South St & Whiteh...|      40.70122128|      -74.01234218|
|2017|     E 43 St & 2 Ave|      40.75022392|      -73.97121414|
| 504|     1 Ave & E 16 St|      40.73221853|      -73.98165557|
|3242|Schermerhorn St &...|40.69102925677968|-73.99183362722397|
|3491|    E 118 St & 1 Ave|         40.79747|         -73.93504|
|3664|North Moore St & ...|40.72019521437465| -74.0103006362915|
|3260|Mercer St & Bleec...|40.72706363348306|-73.99662137031554|
|3059|Pulaski St & Marc...|       40.6933982|        -73.939877|
| 423|     W 54 St & 9 Av

In [122]:
routes = gf.GraphFrame(stations, edges)
routes

GraphFrame(v:[id: int, name: string ... 2 more fields], e:[src: int, dst: int ... 1 more field])

In [123]:
routes.triplets.show()

+--------------------+-----------------+--------------------+
|                 src|             edge|                 dst|
+--------------------+-----------------+--------------------+
|[471, Grand St & ...|   [471, 471, 66]|[471, Grand St & ...|
|[471, Grand St & ...|[471, 471, 13260]|[471, Grand St & ...|
|[471, Grand St & ...|  [471, 471, 582]|[471, Grand St & ...|
|[471, Grand St & ...| [471, 471, 2626]|[471, Grand St & ...|
|[471, Grand St & ...| [471, 471, 2649]|[471, Grand St & ...|
|[471, Grand St & ...|  [471, 471, 870]|[471, Grand St & ...|
|[471, Grand St & ...|  [471, 471, 784]|[471, Grand St & ...|
|[471, Grand St & ...|   [471, 471, 80]|[471, Grand St & ...|
|[471, Grand St & ...|  [471, 471, 997]|[471, Grand St & ...|
|[471, Grand St & ...|   [471, 471, 62]|[471, Grand St & ...|
|[471, Grand St & ...| [471, 471, 1127]|[471, Grand St & ...|
|[471, Grand St & ...|  [471, 471, 198]|[471, Grand St & ...|
|[471, Grand St & ...|  [471, 471, 347]|[471, Grand St & ...|
|[471, G

In [124]:
routes_pr = routes.pageRank(resetProbability=0.1, maxIter=5)

2. Выводим по убыванию

In [125]:
routes_pr.vertices.orderBy(-F.col("pagerank")).show()

+----+--------------------+------------------+------------------+------------------+
|  id|                name|               lat|               lng|          pagerank|
+----+--------------------+------------------+------------------+------------------+
| 519|Pershing Square N...|         40.751873|        -73.977706| 4.406774001083143|
| 402|  Broadway & E 22 St|        40.7403432|      -73.98955109|  3.53228723171532|
| 435|     W 21 St & 6 Ave|       40.74173969|      -73.99415556| 3.269230720393528|
|3255|     8 Ave & W 31 St|  40.7505853470215| -73.9946848154068|3.2361769412840924|
| 497|  E 17 St & Broadway|       40.73704984|      -73.99009296|3.1645640088778486|
| 285|  Broadway & E 14 St|       40.73454567|      -73.99074142|3.0455207021823973|
|3429|Hanson Pl & Ashla...| 40.68506807308177|-73.97790759801863|2.8908249353666955|
| 359|  E 47 St & Park Ave|       40.75510267|      -73.97498696| 2.887482309374865|
| 477|     W 41 St & 8 Ave|       40.75640548|       -73.9900262|

3. Отображаем на тепловой карте

In [126]:
trips_matrix = gf_routes_pr.vertices.toPandas()[["lat", "lng", "pagerank"]].values

In [127]:
m = folium.Map()
HeatMap(trips_matrix, radius=15).add_to(m)
m.fit_bounds(m.get_bounds())
embed_map(m)

## Задача 3

оцените дистанцию поездок (в метрах) на основе координат начальной и конечной станций

In [128]:
from pyspark.sql.functions import acos, cos, sin, lit, toRadians

def haversine(long_x, lat_x, long_y, lat_y):
    return acos(
        sin(toRadians(lat_x)) * sin(toRadians(lat_y)) +
        cos(toRadians(lat_x)) * cos(toRadians(lat_y)) *
        cos(toRadians(long_x) - toRadians(long_y))
    ) * lit(6371000.0)

from pyspark.sql.functions import udf

haversine_udf = udf(haversine, DoubleType())

выведите максимальное, среднее значение, стандартное отклонение и медиан

In [132]:
trips_new = trips.dropna(subset=['start_station_id']).dropna(subset=['end_station_id'])

trips_routes = trips_new\
            .dropDuplicates(['start_station_id','end_station_id'])\
            .select("start_station_name", "end_station_name", 
                    F.col("start_station_latitude"), F.col("start_station_longitude"),
                    F.col("end_station_latitude"), F.col("end_station_longitude"))\
            .withColumn("dist_m", haversine("start_station_longitude",
                                             "start_station_latitude", 
                                             "end_station_longitude",
                                             "end_station_latitude").alias("dist_m"))\
            .filter(F.col("start_station_id") != F.col("end_station_id"))

print("Количество маршрутов: {}".format(trips_routes.count()))
trips_routes.show()

Количество маршрутов: 131259
+--------------------+--------------------+----------------------+-----------------------+--------------------+---------------------+------------------+
|  start_station_name|    end_station_name|start_station_latitude|start_station_longitude|end_station_latitude|end_station_longitude|            dist_m|
+--------------------+--------------------+----------------------+-----------------------+--------------------+---------------------+------------------+
|    W 52 St & 11 Ave|     W 44 St & 5 Ave|           40.76727216|           -73.99392888|         40.75500254|         -73.98014437|1791.4335586355398|
|Barrow St & Hudso...|     W 37 St & 5 Ave|           40.73172428|           -74.00674436|         40.75038009|         -73.98338988|2859.1355781783695|
|Barrow St & Hudso...|  Broadway & W 29 St|           40.73172428|           -74.00674436|          40.7462009|         -73.98855723|2222.4162947640793|
|Hudson St & Reade St|     7 Ave & Park Pl|          

In [134]:
trips_routes.describe("dist_m").show()

+-------+------------------+
|summary|            dist_m|
+-------+------------------+
|  count|            131259|
|   mean|2818.1040340510817|
| stddev| 1813.459185512828|
|    min| 36.73591246092364|
|    max|15326.431486453896|
+-------+------------------+



In [135]:
median = trips_routes.approxQuantile("dist_m", [0.5], 0)[0]
median

2435.3733872294574

In [136]:
max_start_count = trips_routes.select(F.max("dist_m").alias("max")).rdd.collect()[0]["max"]
max_start_count

15326.431486453896

In [137]:
min_start_count = trips_routes.select(F.min("dist_m").alias("min")).rdd.collect()[0]["min"]
min_start_count

36.73591246092364

In [139]:
avg_start_count = trips_routes.select(F.avg("dist_m").alias("avg")).rdd.collect()[0]["avg"]
avg_start_count

2818.1040340510817

## Задача 4
1. определите для каждой станции среднее количество начала поездок и количество завершения поездок:

а) в день

In [141]:
from pyspark.sql.functions  import date_format, unix_timestamp, hour
# для старта поездок. конвертируем в дни, конвертируем в дату, считаем число дней каждого дня недели
# итого в выводе получаем сопоставление станции и количества поездок в определнную дату
trips_routes_days = trips_new\
    .select(F.col('start_station_id').alias('id'), 
            F.col('start_station_name'),
            F.col("starttime"))\
    .withColumn("DayOfWeek", date_format("starttime","E"))\
    .withColumn('starttime_unix', date_format('starttime', "yyyyMMdd"))\
    .groupBy("id", "DayOfWeek", "starttime_unix") \
    .agg(F.first('start_station_name').alias('name'), F.count("DayOfWeek").alias("begin count"))\
    .orderBy('id', 'starttime_unix')

trips_routes_days.show()

# агрегируем полученные данные в нужном формате
print("Среднее количество начала поездок для каждого дня недели:")
trips_routes_days\
    .groupBy("id", "DayOfWeek", 'name')\
    .agg(F.avg("begin count").alias("begin avg"))\
    .orderBy("id", "DayOfWeek").show()

print("Среднее количество начала поездок в день (не привязано к дню недели):")
trips_routes_days\
    .groupBy("id", 'name')\
    .agg(F.avg("begin count").alias("begin avg"))\
    .orderBy("id").show()

+---+---------+--------------+----------------+-----------+
| id|DayOfWeek|starttime_unix|            name|begin count|
+---+---------+--------------+----------------+-----------+
| 72|      Fri|      20190201|W 52 St & 11 Ave|         37|
| 72|      Sat|      20190202|W 52 St & 11 Ave|         22|
| 72|      Sun|      20190203|W 52 St & 11 Ave|         57|
| 72|      Mon|      20190204|W 52 St & 11 Ave|        100|
| 72|      Tue|      20190205|W 52 St & 11 Ave|        109|
| 72|      Wed|      20190206|W 52 St & 11 Ave|         87|
| 72|      Thu|      20190207|W 52 St & 11 Ave|         64|
| 72|      Fri|      20190208|W 52 St & 11 Ave|         90|
| 72|      Sat|      20190209|W 52 St & 11 Ave|         53|
| 72|      Sun|      20190210|W 52 St & 11 Ave|         45|
| 72|      Mon|      20190211|W 52 St & 11 Ave|        102|
| 72|      Tue|      20190212|W 52 St & 11 Ave|         39|
| 72|      Wed|      20190213|W 52 St & 11 Ave|         60|
| 72|      Thu|      20190214|W 52 St & 

In [143]:
# для старта поездок. конвертируем в дни, конвертируем в дату, считаем число дней каждого дня недели
# итого в выводе получаем сопоставление станции и количества поездок в определнную дату
trips_routes_days = trips_new\
    .select(F.col('end_station_id').alias('id'), 
            F.col('end_station_name'),
            F.col("stoptime"))\
    .withColumn("DayOfWeek", date_format("stoptime","E"))\
    .withColumn('stoptime_unix', date_format('stoptime', "yyyyMMdd"))\
    .groupBy("id", "DayOfWeek", "stoptime_unix") \
    .agg(F.first('end_station_name').alias('name'), F.count("DayOfWeek").alias("end count"))\
    .orderBy('id', 'stoptime_unix')

trips_routes_days.show()

# агрегируем полученные данные в нужном формате
print("Среднее количество окончания поездок для каждого дня недели:")
trips_routes_days\
    .groupBy("id", "DayOfWeek", 'name')\
    .agg(F.avg("end count").alias("end_avg"))\
    .orderBy("id", "DayOfWeek").show()

print("Среднее количество окончания поездок в день (не привязано к дню недели):")
trips_routes_days\
    .groupBy("id", 'name')\
    .agg(F.avg("end count").alias("end_avg"))\
    .orderBy("id").show()

+---+---------+-------------+----------------+---------+
| id|DayOfWeek|stoptime_unix|            name|end count|
+---+---------+-------------+----------------+---------+
| 72|      Fri|     20190201|W 52 St & 11 Ave|       37|
| 72|      Sat|     20190202|W 52 St & 11 Ave|       25|
| 72|      Sun|     20190203|W 52 St & 11 Ave|       55|
| 72|      Mon|     20190204|W 52 St & 11 Ave|      122|
| 72|      Tue|     20190205|W 52 St & 11 Ave|      104|
| 72|      Wed|     20190206|W 52 St & 11 Ave|       73|
| 72|      Thu|     20190207|W 52 St & 11 Ave|       77|
| 72|      Fri|     20190208|W 52 St & 11 Ave|       84|
| 72|      Sat|     20190209|W 52 St & 11 Ave|       71|
| 72|      Sun|     20190210|W 52 St & 11 Ave|       36|
| 72|      Mon|     20190211|W 52 St & 11 Ave|       94|
| 72|      Tue|     20190212|W 52 St & 11 Ave|       39|
| 72|      Wed|     20190213|W 52 St & 11 Ave|       64|
| 72|      Thu|     20190214|W 52 St & 11 Ave|       77|
| 72|      Fri|     20190215|W 

б) утром (06:00-11:59), днем (12:00-17:59), вечером (18:00-23:59), ночью (00:00-05:59)

In [144]:
# выведем следнее количество по часам, вообще не привязываясь к дням недели
trips_routes_hours_begin = trips_new.select(F.col('start_station_id').alias('id'), 
                               F.col('start_station_name').alias('name'),
                            F.col("starttime"), 
                            hour(F.col('starttime')).alias('start_hour'))\
            .withColumn('starttime_unix', date_format('starttime', "yyyyMMdd"))\
            .groupBy('id','start_hour','name','starttime_unix')\
            .agg(F.count('start_hour').alias('begin count'))\
            .orderBy('id', 'starttime_unix', 'start_hour')
trips_routes_hours_begin.show()
print("Среднее число начала поездок в течение дня:")

print("Ночью:")
trips_routes_hours_begin.where(F.col('start_hour') < 6).where(F.col('start_hour') >= 0)\
        .groupBy('id','name').agg(F.avg('begin count').alias('time 0-6')).orderBy('id').show()
print("Утром:")
trips_routes_hours_begin.where(F.col('start_hour') < 12).where(F.col('start_hour') >= 6)\
        .groupBy('id','name').agg(F.avg('begin count').alias('time 6-12')).orderBy('id').show()
print("Днем:")
trips_routes_hours_begin.where(F.col('start_hour') < 18).where(F.col('start_hour') >= 12)\
        .groupBy('id','name').agg(F.avg('begin count').alias('time 12-18')).orderBy('id').show()
print("Вечером:")
trips_routes_hours_begin.where(F.col('start_hour') < 24).where(F.col('start_hour') >= 18)\
        .groupBy('id','name').agg(F.avg('begin count').alias('time 18-24')).orderBy('id').show()

+---+----------+----------------+--------------+-----------+
| id|start_hour|            name|starttime_unix|begin count|
+---+----------+----------------+--------------+-----------+
| 72|         7|W 52 St & 11 Ave|      20190201|          1|
| 72|         8|W 52 St & 11 Ave|      20190201|          4|
| 72|         9|W 52 St & 11 Ave|      20190201|          4|
| 72|        10|W 52 St & 11 Ave|      20190201|          2|
| 72|        11|W 52 St & 11 Ave|      20190201|          4|
| 72|        12|W 52 St & 11 Ave|      20190201|          4|
| 72|        13|W 52 St & 11 Ave|      20190201|          2|
| 72|        14|W 52 St & 11 Ave|      20190201|          5|
| 72|        15|W 52 St & 11 Ave|      20190201|          4|
| 72|        16|W 52 St & 11 Ave|      20190201|          2|
| 72|        17|W 52 St & 11 Ave|      20190201|          1|
| 72|        18|W 52 St & 11 Ave|      20190201|          2|
| 72|        19|W 52 St & 11 Ave|      20190201|          1|
| 72|        21|W 52 St 

In [146]:
# выведем следнее количество по часам, вообще не привязываясь к дням недели
trips_routes_hours_end = trips_new\
            .select(F.col('end_station_id').alias('id'), 
                    F.col('end_station_name').alias('name'),
                    F.col("stoptime"), 
                    hour(F.col('stoptime')).alias('stop_hour'))\
            .withColumn('stoptime_unix', date_format('stoptime', "yyyyMMdd"))\
            .groupBy('id','stop_hour','name','stoptime_unix')\
            .agg(F.count('stop_hour').alias('end count'))\
            .orderBy('id', 'stoptime_unix', 'stop_hour')
trips_routes_hours_end.show()
print("Среднее число окончания поездок в течение дня:")

print("Ночью:")
trips_routes_hours_end.filter(F.col('stop_hour') < 6).filter(F.col('stop_hour') >= 0)\
        .groupBy('id','name').agg(F.avg('end count').alias('time 0-6')).orderBy('id').show()
print("Утром:")
trips_routes_hours_end.filter(F.col('stop_hour') < 12).filter(F.col('stop_hour') >= 6)\
        .groupBy('id','name').agg(F.avg('end count').alias('time 6-12')).orderBy('id').show()
print("Днем:")
trips_routes_hours_end.filter(F.col('stop_hour') < 18).filter(F.col('stop_hour') >= 12)\
        .groupBy('id','name').agg(F.avg('end count').alias('time 12-18')).orderBy('id').show()
print("Вечером:")
trips_routes_hours_end.filter(F.col('stop_hour') < 24).filter(F.col('stop_hour') >= 18)\
        .groupBy('id','name').agg(F.avg('end count').alias('time 18-24')).orderBy('id').show()

+---+---------+----------------+-------------+---------+
| id|stop_hour|            name|stoptime_unix|end count|
+---+---------+----------------+-------------+---------+
| 72|        6|W 52 St & 11 Ave|     20190201|        2|
| 72|        7|W 52 St & 11 Ave|     20190201|        1|
| 72|        8|W 52 St & 11 Ave|     20190201|        1|
| 72|        9|W 52 St & 11 Ave|     20190201|        4|
| 72|       10|W 52 St & 11 Ave|     20190201|        6|
| 72|       11|W 52 St & 11 Ave|     20190201|        1|
| 72|       12|W 52 St & 11 Ave|     20190201|        2|
| 72|       14|W 52 St & 11 Ave|     20190201|        5|
| 72|       15|W 52 St & 11 Ave|     20190201|        3|
| 72|       16|W 52 St & 11 Ave|     20190201|        1|
| 72|       17|W 52 St & 11 Ave|     20190201|        1|
| 72|       18|W 52 St & 11 Ave|     20190201|        4|
| 72|       19|W 52 St & 11 Ave|     20190201|        2|
| 72|       21|W 52 St & 11 Ave|     20190201|        1|
| 72|       22|W 52 St & 11 Ave

в) в среду и в воскресенье по временным диапазонам (см. выше)

In [147]:
# выделяем дни со средой и субботой
trips_routes_hours_sun_wed_start = df_trips_new\
            .withColumn('start_dayofweek', date_format(F.col('starttime'), 'E'))\
            .withColumn('starttime_unix', date_format('starttime', "yyyyMMdd"))\
            .where((F.col('start_dayofweek') == 'Sun') | (F.col('start_dayofweek') == 'Wed'))
trips_routes_hours_sun_wed_start.show()

trips_routes_hours_sun_wed_stop = df_trips_new\
            .withColumn('stop_dayofweek', date_format(F.col('stoptime'), 'E'))\
            .withColumn('stop_unix', date_format('stoptime', "yyyyMMdd"))\
            .where((F.col('stop_dayofweek') == 'Sun') | (F.col('stop_dayofweek') == 'Wed'))
trips_routes_hours_sun_wed_stop.show()

+------------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+---------------+--------------+
|tripduration|           starttime|            stoptime|start station id|  start station name|start station latitude|start station longitude|end station id|    end station name|end station latitude|end station longitude|bikeid|  usertype|birth year|gender|start_dayofweek|starttime_unix|
+------------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+---------------+--------------+
|         366|2019-02-03 00:00:...|2019-02-03 00:06:...|             439|      E 4 St & 2 Ave|            40.7262807|           -73.9897

In [152]:
# выведем следнее количество по часам для среды
trips_routes_hours_begin_sw = trips_routes_hours_sun_wed_start\
            .select(F.col('start station id').alias('id'), 
                    F.col('start station name').alias('name'),
                    F.col("starttime"), 
                    F.col("start_dayofweek"),
                    hour(F.col('starttime')).alias('start_hour'))\
            .withColumn('starttime_unix', date_format('starttime', "yyyyMMdd"))\
            .groupBy('id','start_hour','name','starttime_unix', 'start_dayofweek')\
            .agg(F.count('start_hour').alias('begin count'))\
            .orderBy('id', 'starttime_unix', 'start_hour', "start_dayofweek")
trips_routes_hours_begin_sw.show()
print("Среднее число начала поездок в течение среды:")

print("Ночью:")
trips_routes_hours_begin_sw.where(F.col("start_dayofweek") == "Wed")\
        .where(F.col('start_hour') < 6).where(F.col('start_hour') >= 0)\
        .groupBy('id','name').agg(F.avg('begin count').alias('time 0-6')).orderBy('id').show()
print("Утром:")
trips_routes_hours_begin_sw.where(F.col("start_dayofweek") == "Wed")\
        .where(F.col('start_hour') < 12).where(F.col('start_hour') >= 6)\
        .groupBy('id','name').agg(F.avg('begin count').alias('time 6-12')).orderBy('id').show()
print("Днем:")
trips_routes_hours_begin_sw.where(F.col("start_dayofweek") == "Wed")\
        .where(F.col('start_hour') < 18).where(F.col('start_hour') >= 12)\
        .groupBy('id','name').agg(F.avg('begin count').alias('time 12-18')).orderBy('id').show()
print("Вечером:")
trips_routes_hours_begin_sw.where(F.col("start_dayofweek") == "Wed")\
        .where(F.col('start_hour') < 24).where(F.col('start_hour') >= 18)\
        .groupBy('id','name').agg(F.avg('begin count').alias('time 18-24')).orderBy('id').show()

+---+----------+----------------+--------------+---------------+-----------+
| id|start_hour|            name|starttime_unix|start_dayofweek|begin count|
+---+----------+----------------+--------------+---------------+-----------+
| 72|         0|W 52 St & 11 Ave|      20190203|            Sun|          1|
| 72|         1|W 52 St & 11 Ave|      20190203|            Sun|          1|
| 72|         5|W 52 St & 11 Ave|      20190203|            Sun|          1|
| 72|         8|W 52 St & 11 Ave|      20190203|            Sun|          5|
| 72|         9|W 52 St & 11 Ave|      20190203|            Sun|          2|
| 72|        10|W 52 St & 11 Ave|      20190203|            Sun|          1|
| 72|        11|W 52 St & 11 Ave|      20190203|            Sun|          2|
| 72|        12|W 52 St & 11 Ave|      20190203|            Sun|          1|
| 72|        13|W 52 St & 11 Ave|      20190203|            Sun|          3|
| 72|        14|W 52 St & 11 Ave|      20190203|            Sun|         12|

In [153]:
# выведем следнее количество по часам для воскресенья
print("Среднее число начала поездок в течение воскресенья:")

print("Ночью:")
trips_routes_hours_begin_sw.where(F.col("start_dayofweek") == "Sun")\
        .where(F.col('start_hour') < 6).where(F.col('start_hour') >= 0)\
        .groupBy('id','name').agg(F.avg('begin count').alias('time 0-6')).orderBy('id').show()
print("Утром:")
trips_routes_hours_begin_sw.where(F.col("start_dayofweek") == "Sun")\
        .where(F.col('start_hour') < 12).where(F.col('start_hour') >= 6)\
        .groupBy('id','name').agg(F.avg('begin count').alias('time 6-12')).orderBy('id').show()
print("Днем:")
trips_routes_hours_begin_sw.where(F.col("start_dayofweek") == "Sun")\
        .where(F.col('start_hour') < 18).where(F.col('start_hour') >= 12)\
        .groupBy('id','name').agg(F.avg('begin count').alias('time 12-18')).orderBy('id').show()
print("Вечером:")
trips_routes_hours_begin_sw.where(F.col("start_dayofweek") == "Sun")\
        .where(F.col('start_hour') < 24).where(F.col('start_hour') >= 18)\
        .groupBy('id','name').agg(F.avg('begin count').alias('time 18-24')).orderBy('id').show()

Среднее число начала поездок в течение воскресенья:
Ночью:
+---+--------------------+------------------+
| id|                name|          time 0-6|
+---+--------------------+------------------+
| 72|    W 52 St & 11 Ave|               1.0|
| 79|Franklin St & W B...|1.5714285714285714|
| 82|St James Pl & Pea...|               1.0|
| 83|Atlantic Ave & Fo...|               1.0|
|119|Park Ave & St Edw...|               1.0|
|120|Lexington Ave & C...|               1.2|
|127|Barrow St & Hudso...|               1.5|
|128|MacDougal St & Pr...|1.8571428571428572|
|143|Clinton St & Jora...|               1.0|
|144| Nassau St & Navy St|               1.0|
|146|Hudson St & Reade St|1.6666666666666667|
|150|   E 2 St & Avenue C|               1.2|
|151|Cleveland Pl & Sp...|               1.8|
|157|Henry St & Atlant...|               2.0|
|161|LaGuardia Pl & W ...|2.4444444444444446|
|164|     E 47 St & 2 Ave|               1.0|
|167|     E 39 St & 3 Ave|               1.0|
|168|     W 18 St & 6

In [156]:
# выведем следнее количество по часам для среды
trips_routes_hours_end_sw = trips_routes_hours_sun_wed_stop\
            .select(F.col('end station id').alias('id'), 
                               F.col('end station name').alias('name'),
                            F.col("stoptime"), 
                            F.col("stop_dayofweek"),
                            hour(F.col('stoptime')).alias('stop_hour'))\
            .withColumn('stoptime_unix', date_format('stoptime', "yyyyMMdd"))\
            .groupBy('id','stop_hour','name','stoptime_unix', 'stop_dayofweek')\
            .agg(F.count('stop_hour').alias('end count'))\
            .orderBy('id', 'stoptime_unix', 'stop_hour', "stop_dayofweek")
trips_routes_hours_end_sw.show()
print("Среднее число конца поездок в течение среды:")

print("Ночью:")
trips_routes_hours_end_sw.where(F.col("stop_dayofweek") == "Wed")\
        .where(F.col('stop_hour') < 6).where(F.col('stop_hour') >= 0)\
        .groupBy('id','name').agg(F.avg('end count').alias('time 0-6')).orderBy('id').show()
print("Утром:")
trips_routes_hours_end_sw.where(F.col("stop_dayofweek") == "Wed")\
        .where(F.col('stop_hour') < 12).where(F.col('stop_hour') >= 6)\
        .groupBy('id','name').agg(F.avg('end count').alias('time 6-12')).orderBy('id').show()
print("Днем:")
trips_routes_hours_end_sw.where(F.col("stop_dayofweek") == "Wed")\
        .where(F.col('stop_hour') < 18).where(F.col('stop_hour') >= 12)\
        .groupBy('id','name').agg(F.avg('end count').alias('time 12-18')).orderBy('id').show()
print("Вечером:")
trips_routes_hours_end_sw.where(F.col("stop_dayofweek") == "Wed")\
        .where(F.col('stop_hour') < 24).where(F.col('stop_hour') >= 18)\
        .groupBy('id','name').agg(F.avg('end count').alias('time 18-24')).orderBy('id').show()

+---+---------+----------------+-------------+--------------+---------+
| id|stop_hour|            name|stoptime_unix|stop_dayofweek|end count|
+---+---------+----------------+-------------+--------------+---------+
| 72|        0|W 52 St & 11 Ave|     20190203|           Sun|        1|
| 72|        9|W 52 St & 11 Ave|     20190203|           Sun|        1|
| 72|       10|W 52 St & 11 Ave|     20190203|           Sun|        1|
| 72|       11|W 52 St & 11 Ave|     20190203|           Sun|        1|
| 72|       12|W 52 St & 11 Ave|     20190203|           Sun|        4|
| 72|       13|W 52 St & 11 Ave|     20190203|           Sun|        8|
| 72|       14|W 52 St & 11 Ave|     20190203|           Sun|       11|
| 72|       15|W 52 St & 11 Ave|     20190203|           Sun|        9|
| 72|       16|W 52 St & 11 Ave|     20190203|           Sun|        6|
| 72|       17|W 52 St & 11 Ave|     20190203|           Sun|        4|
| 72|       18|W 52 St & 11 Ave|     20190203|           Sun|   

In [157]:
print("Среднее число конца поездок в течение воскресенья:")

print("Ночью:")
trips_routes_hours_end_sw.where(F.col("stop_dayofweek") == "Sun")\
        .where(F.col('stop_hour') < 6).where(F.col('stop_hour') >= 0)\
        .groupBy('id','name').agg(F.avg('end count').alias('time 0-6')).orderBy('id').show()
print("Утром:")
trips_routes_hours_end_sw.where(F.col("stop_dayofweek") == "Sun")\
        .where(F.col('stop_hour') < 12).where(F.col('stop_hour') >= 6)\
        .groupBy('id','name').agg(F.avg('end count').alias('time 6-12')).orderBy('id').show()
print("Днем:")
trips_routes_hours_end_sw.where(F.col("stop_dayofweek") == "Sun")\
        .where(F.col('stop_hour') < 18).where(F.col('stop_hour') >= 12)\
        .groupBy('id','name').agg(F.avg('end count').alias('time 12-18')).orderBy('id').show()
print("Вечером:")
trips_routes_hours_end_sw.where(F.col("stop_dayofweek") == "Sun")\
        .where(F.col('stop_hour') < 24).where(F.col('stop_hour') >= 18)\
        .groupBy('id','name').agg(F.avg('end count').alias('time 18-24')).orderBy('id').show()

Среднее число конца поездок в течение воскресенья:
Ночью:
+---+--------------------+------------------+
| id|                name|          time 0-6|
+---+--------------------+------------------+
| 72|    W 52 St & 11 Ave|               1.0|
| 79|Franklin St & W B...|               1.0|
| 82|St James Pl & Pea...|               1.2|
| 83|Atlantic Ave & Fo...|               1.0|
|120|Lexington Ave & C...|               1.4|
|127|Barrow St & Hudso...|1.3333333333333333|
|128|MacDougal St & Pr...|1.4285714285714286|
|143|Clinton St & Jora...|               1.0|
|144| Nassau St & Navy St|               1.0|
|146|Hudson St & Reade St|               1.0|
|150|   E 2 St & Avenue C|1.9166666666666667|
|151|Cleveland Pl & Sp...|1.6666666666666667|
|157|Henry St & Atlant...|1.3333333333333333|
|161|LaGuardia Pl & W ...|1.1428571428571428|
|164|     E 47 St & 2 Ave|1.1428571428571428|
|167|     E 39 St & 3 Ave|               1.5|
|168|     W 18 St & 6 Ave|               1.0|
|173|  Broadway & W 49

In [162]:
# выведем среднее количество по часам, вообще не привязываясь к дням недели
# для начала поездок
trips_routes_hours_begin = trips_new.select(F.col('start_station_id').alias('id'), 
                               F.col('start_station_name').alias('name'),
                            F.col("starttime"), 
                            F.col("start_station_latitude").alias('lat'), 
                            F.col("start_station_longitude").alias('lng'),
                            hour(F.col('starttime')).alias('start_hour'))\
            .withColumn('starttime_unix', date_format('starttime', "yyyyMMdd"))\
            .groupBy('id','start_hour','name','starttime_unix', 'lat', 'lng')\
            .agg(F.count('start_hour').alias('begin count'))\
            .orderBy('id', 'starttime_unix', 'start_hour')

trips_routes_hours_begin.show()
print("Среднее число начала поездок в течение дня:")

print("Ночью:")
trips_routes_hours_begin_night = trips_routes_hours_begin\
        .where(F.col('start_hour') < 6).where(F.col('start_hour') >= 0)\
        .groupBy('id','name', 'lat', 'lng')\
        .agg(F.avg('begin count').alias('time 0-6')).orderBy('id')
trips_routes_hours_begin_night.show()

print("Утром:")
trips_routes_hours_begin_morning = trips_routes_hours_begin\
        .where(F.col('start_hour') < 12).where(F.col('start_hour') >= 6)\
        .groupBy('id','name','lat','lng')\
        .agg(F.avg('begin count').alias('time 6-12')).orderBy('id')
trips_routes_hours_begin_morning.show()

print("Днем:")
trips_routes_hours_begin_afternoon = trips_routes_hours_begin\
        .where(F.col('start_hour') < 18).where(F.col('start_hour') >= 12)\
        .groupBy('id','name','lat','lng')\
        .agg(F.avg('begin count').alias('time 12-18')).orderBy('id')
trips_routes_hours_begin_afternoon.show()

print("Вечером:")
trips_routes_hours_begin_evening = trips_routes_hours_begin\
        .where(F.col('start_hour') < 24).where(F.col('start_hour') >= 18)\
        .groupBy('id','name', 'lat', 'lng')\
        .agg(F.avg('begin count').alias('time 18-24')).orderBy('id')
trips_routes_hours_begin_evening.show()

+---+----------+----------------+--------------+-----------+------------+-----------+
| id|start_hour|            name|starttime_unix|        lat|         lng|begin count|
+---+----------+----------------+--------------+-----------+------------+-----------+
| 72|         7|W 52 St & 11 Ave|      20190201|40.76727216|-73.99392888|          1|
| 72|         8|W 52 St & 11 Ave|      20190201|40.76727216|-73.99392888|          4|
| 72|         9|W 52 St & 11 Ave|      20190201|40.76727216|-73.99392888|          4|
| 72|        10|W 52 St & 11 Ave|      20190201|40.76727216|-73.99392888|          2|
| 72|        11|W 52 St & 11 Ave|      20190201|40.76727216|-73.99392888|          4|
| 72|        12|W 52 St & 11 Ave|      20190201|40.76727216|-73.99392888|          4|
| 72|        13|W 52 St & 11 Ave|      20190201|40.76727216|-73.99392888|          2|
| 72|        14|W 52 St & 11 Ave|      20190201|40.76727216|-73.99392888|          5|
| 72|        15|W 52 St & 11 Ave|      20190201|40.767

+---+--------------------+------------------+------------------+------------------+
| id|                name|               lat|               lng|        time 18-24|
+---+--------------------+------------------+------------------+------------------+
| 72|    W 52 St & 11 Ave|       40.76727216|      -73.99392888|3.4414414414414414|
| 79|Franklin St & W B...|       40.71911552|      -74.00666661|3.2916666666666665|
| 82|St James Pl & Pea...|       40.71117416|      -74.00016545| 1.941860465116279|
| 83|Atlantic Ave & Fo...|       40.68382604|      -73.97632328|2.3229166666666665|
|119|Park Ave & St Edw...|       40.69608941|      -73.97803415|1.2105263157894737|
|120|Lexington Ave & C...|       40.68676793|      -73.95928168|1.4736842105263157|
|127|Barrow St & Hudso...|       40.73172428|      -74.00674436|             5.312|
|128|MacDougal St & Pr...|       40.72710258|      -74.00297088|4.8311688311688314|
|143|Clinton St & Jora...|       40.69239502|      -73.99337909| 3.107142857

In [166]:
# выведем среднее количество по часам, вообще не привязываясь к дням недели
# для окончания поездок
trips_routes_hours_end = trips_new.select(F.col('end_station_id').alias('id'), 
                               F.col('end_station_name').alias('name'),
                            F.col("stoptime"), 
                            F.col("end_station_latitude").alias('lat'), 
                            F.col("end_station_longitude").alias('lng'),
                            hour(F.col('stoptime')).alias('stop_hour'))\
            .withColumn('stoptime_unix', date_format('stoptime', "yyyyMMdd"))\
            .groupBy('id','stop_hour','name','stoptime_unix','lat','lng')\
            .agg(F.count('stop_hour').alias('end count'))\
            .orderBy('id', 'stoptime_unix', 'stop_hour')
trips_routes_hours_end.show()
print("Среднее число окончания поездок в течение дня:")

print("Ночью:")
trips_routes_hours_end_night = trips_routes_hours_end\
        .where(F.col('stop_hour') < 6).where(F.col('stop_hour') >= 0)\
        .groupBy('id','name', 'lat','lng')\
        .agg(F.avg('end count').alias('time 0-6')).orderBy('id')
trips_routes_hours_end_night.show()

print("Утром:")
trips_routes_hours_end_morning = trips_routes_hours_end\
        .where(F.col('stop_hour') < 12).where(F.col('stop_hour') >= 6)\
        .groupBy('id','name','lat','lng')\
        .agg(F.avg('end count').alias('time 6-12')).orderBy('id')
trips_routes_hours_end_morning.show()

print("Днем:")
trips_routes_hours_end_afternoon = trips_routes_hours_end\
        .where(F.col('stop_hour') < 18).where(F.col('stop_hour') >= 12)\
        .groupBy('id','name','lat','lng')\
        .agg(F.avg('end count').alias('time 12-18')).orderBy('id')
trips_routes_hours_end_afternoon.show()

print("Вечером:")
trips_routes_hours_end_evening = trips_routes_hours_end\
        .where(F.col('stop_hour') < 24).where(F.col('stop_hour') >= 18)\
        .groupBy('id','name','lat','lng')\
        .agg(F.avg('end count').alias('time 18-24')).orderBy('id')
trips_routes_hours_end_evening.show()

+---+---------+----------------+-------------+-----------+------------+---------+
| id|stop_hour|            name|stoptime_unix|        lat|         lng|end count|
+---+---------+----------------+-------------+-----------+------------+---------+
| 72|        6|W 52 St & 11 Ave|     20190201|40.76727216|-73.99392888|        2|
| 72|        7|W 52 St & 11 Ave|     20190201|40.76727216|-73.99392888|        1|
| 72|        8|W 52 St & 11 Ave|     20190201|40.76727216|-73.99392888|        1|
| 72|        9|W 52 St & 11 Ave|     20190201|40.76727216|-73.99392888|        4|
| 72|       10|W 52 St & 11 Ave|     20190201|40.76727216|-73.99392888|        6|
| 72|       11|W 52 St & 11 Ave|     20190201|40.76727216|-73.99392888|        1|
| 72|       12|W 52 St & 11 Ave|     20190201|40.76727216|-73.99392888|        2|
| 72|       14|W 52 St & 11 Ave|     20190201|40.76727216|-73.99392888|        5|
| 72|       15|W 52 St & 11 Ave|     20190201|40.76727216|-73.99392888|        3|
| 72|       16|W

+---+--------------------+------------------+------------------+------------------+
| id|                name|               lat|               lng|        time 18-24|
+---+--------------------+------------------+------------------+------------------+
| 72|    W 52 St & 11 Ave|       40.76727216|      -73.99392888|  4.01418439716312|
| 79|Franklin St & W B...|       40.71911552|      -74.00666661| 2.127450980392157|
| 82|St James Pl & Pea...|       40.71117416|      -74.00016545|2.1458333333333335|
| 83|Atlantic Ave & Fo...|       40.68382604|      -73.97632328|1.9285714285714286|
|119|Park Ave & St Edw...|       40.69608941|      -73.97803415|1.2647058823529411|
|120|Lexington Ave & C...|       40.68676793|      -73.95928168| 2.017391304347826|
|127|Barrow St & Hudso...|       40.73172428|      -74.00674436| 5.179856115107913|
|128|MacDougal St & Pr...|       40.72710258|      -74.00297088|             5.125|
|143|Clinton St & Jora...|       40.69239502|      -73.99337909| 2.393939393

2. отобразите полученные данные для второго случая в виде тепловой временной карты (HeatMapWithTime)

In [163]:
# сначала построим карту для среднего числа начала поездок
data_begin = []

data_begin.append(trips_routes_hours_begin_night.select(F.col('lat'),F.col('lng'),
                                 F.col('time 0-6')).toPandas().values.tolist())
data_begin.append(trips_routes_hours_begin_morning.select(F.col('lat'),F.col('lng'),
                                 F.col('time 6-12')).toPandas().values.tolist())
data_begin.append(trips_routes_hours_begin_afternoon.select(F.col('lat'),F.col('lng'),
                                 F.col('time 12-18')).toPandas().values.tolist())
data_begin.append(trips_routes_hours_begin_evening.select(F.col('lat'),F.col('lng'),
                                 F.col('time 18-24')).toPandas().values.tolist())

[[[40.76727216, -73.99392888, 1.1568627450980393],
  [40.71911552, -74.00666661, 1.2916666666666667],
  [40.71117416, -74.00016545, 1.0909090909090908],
  [40.68382604, -73.97632328, 1.2222222222222223],
  [40.69608941, -73.97803415, 1.0],
  [40.68676793, -73.95928168, 1.0714285714285714],
  [40.73172428, -74.00674436, 1.1666666666666667],
  [40.72710258, -74.00297088, 1.4193548387096775],
  [40.69239502, -73.99337909, 1.1666666666666667],
  [40.69839895, -73.98068914, 1.1666666666666667],
  [40.71625008, -74.0091059, 1.3272727272727274],
  [40.7208736, -73.98085795, 1.2432432432432432],
  [40.722103786686034, -73.99724900722504, 1.4166666666666667],
  [40.69089272, -73.99612349, 1.3333333333333333],
  [40.72917025, -73.99810231, 1.588235294117647],
  [40.75323098, -73.97032517, 1.36],
  [40.7489006, -73.97604882, 1.15625],
  [40.73971301, -73.99456405, 1.1538461538461537],
  [40.76068327096592, -73.9845272898674, 1.25],
  [40.7381765, -73.97738662, 1.1612903225806452],
  [40.70905623,

In [164]:
m = folium.Map()
HeatMapWithTime(data_begin).add_to(m)
m.fit_bounds(m.get_bounds())
embed_map(m)

In [167]:
# построим карту для среднего числа конца поездок
data_end = []

data_end.append(trips_routes_hours_end_night.select(F.col('lat'),F.col('lng'),
                                 F.col('time 0-6')).toPandas().values.tolist())
data_end.append(trips_routes_hours_end_morning.select(F.col('lat'),F.col('lng'),
                                 F.col('time 6-12')).toPandas().values.tolist())
data_end.append(trips_routes_hours_end_afternoon.select(F.col('lat'),F.col('lng'),
                                 F.col('time 12-18')).toPandas().values.tolist())
data_end.append(trips_routes_hours_end_evening.select(F.col('lat'),F.col('lng'),
                                 F.col('time 18-24')).toPandas().values.tolist())

In [168]:
m = folium.Map()
HeatMapWithTime(data_end).add_to(m)
m.fit_bounds(m.get_bounds())
embed_map(m)